In [2]:
import os
import rasterio
from rasterio.plot import reshape_as_image
import rasterio.mask
from rasterio.features import rasterize
import pandas as pd
import geopandas as gpd
from shapely.geometry import mapping, Point, Polygon
from shapely.ops import cascaded_union
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from geopandas import GeoSeries
from shapely.geometry import Polygon
from rasterio.windows import Window
from rasterio.plot import reshape_as_image
import keras
from keras import backend as K
from keras import layers
from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.convolutional import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.optimizers import Adam
from keras.losses import binary_crossentropy
from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

%matplotlib inline

2 most usefull types of satellite imagery:

* Sentinel-2
    
    Max - 10 meters per pixel
    
    Download tiles: https://scihub.copernicus.eu/dhus/#/home




* Landsat-8

    Max - 30 meters per pixel
    
    Download tiles: https://earthexplorer.usgs.gov/

Reading Raster with rasterio

In [ ]:
#read image

In [ ]:
#check shape

In [ ]:
#check meta

In [ ]:
#plot image

In [ ]:
#read train 

In [ ]:
#visualize polygon

In [ ]:
#convert GeoDataframe to raster CRS

In [ ]:
#prepare binary mask

In [3]:
bin_mask_meta = src.meta.copy()
bin_mask_meta.update({'count': 1})
with rasterio.open("/home/quantum/train/train/train.jp2", 'w', **bin_mask_meta) as dst:
    dst.write(mask, 1)

NameError: name 'src' is not defined

In [ ]:
with rasterio.open('/home/quantum/train/train/train.jp2', "r", driver='JP2OpenJPEG') as src:
    raster_image = src.read()
    raster_meta = src.meta

mask = mask.reshape((10980, 10980))
mask = mask.astype(np.uint8)
plt.figure(figsize=(10,10))
plt.imshow(mask)

In [ ]:
def split_image(image_path, save_path, width, height):
    
    img_names = []
    
    if not os.path.exists(save_path):
        os.makedirs(save_path, exist_ok=True)
        print('Data directory created.')

    os.makedirs(f'{save_path}/images', exist_ok=True)
    with rasterio.open(image_path) as src:
        for j in tqdm(range(0, src.height // height)):
            for i in range(0, src.width // width):
                raster_window = src.read(
                    window=Window(i * width, j * height, width, height)
                )
                image_array = reshape_as_image(raster_window)[:, :, :3]
                if np.count_nonzero(image_array) > image_array.size * 0.9:
                    filename_w_ext = os.path.basename(image_path)
                    filename, _ = os.path.splitext(filename_w_ext)
                    image_format = 'tif'
                    piece_name = f'{filename}_{j}_{i}.{image_format}'

                    image_array = reshape_as_image(raster_window)
                    meta = src.meta
                    meta['height'] = image_array.shape[0]
                    meta['width'] = image_array.shape[1]
                    img_names.append(f'{save_path}/images/{piece_name}')
                    with rasterio.open(f'{save_path}/images/{piece_name}', 'w', **meta) as dst:
                        for ix in range(image_array.shape[2]):
                            dst.write(image_array[:, :, ix], ix + 1)

    return img_names

In [ ]:
#split mask

In [ ]:
image_names = split_image('/home/quantum/S2A_MSIL1C_20170101T082332_N0204_R121_T34JEP_20170101T084543.SAFE/GRANULE/L1C_T34JEP_A007983_20170101T084543/IMG_DATA/T34JEP_20170101T082332_TCI.jp2', 
                   '/home/quantum/split', 256, 256)

In [ ]:
mask_names, drop_list = split_mask('/home/quantum/train/train/train.jp2', 
                   '/home/quantum/split', 256, 256)

In [ ]:
len(drop_list)

In [ ]:
#select used names

Unet architecture

![title](unet.png)

In [ ]:
def build_model(input_shape):
    inputs = Input(input_shape)

    conv1 = Conv2D(8, (3, 3), activation = 'relu', padding = 'same')(inputs)
    conv1 = Conv2D(8, (3, 3), activation = 'relu', padding = 'same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = Conv2D(16, (3, 3), activation = 'relu', padding = 'same')(pool1)
    conv2 = Conv2D(16, (3, 3), activation = 'relu', padding = 'same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    conv3 = Conv2D(32, (3, 3), activation = 'relu', padding = 'same')(pool2)
    conv3 = Conv2D(32, (3, 3), activation = 'relu', padding = 'same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    conv4 = Conv2D(64, (3, 3), activation = 'relu', padding = 'same')(pool3)
    conv4 = Conv2D(64, (3, 3), activation = 'relu', padding = 'same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(128, (3, 3), activation = 'relu', padding = 'same')(pool4)
    conv5 = Conv2D(128, (3, 3), activation = 'relu', padding = 'same')(conv5)

    up6 = Conv2D(64, (2, 2), activation = 'relu', padding = 'same')(UpSampling2D(size = (2,2))(conv5))
    merge6 = concatenate([conv4,up6], axis = 3)
    conv6 = Conv2D(64, (3, 3), activation = 'relu', padding = 'same')(merge6)
    conv6 = Conv2D(64, (3, 3), activation = 'relu', padding = 'same')(conv6)

    up7 = Conv2D(32, (2, 2), activation = 'relu', padding = 'same')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(32, (3, 3), activation = 'relu', padding = 'same')(merge7)
    conv7 = Conv2D(32, (3, 3), activation = 'relu', padding = 'same')(conv7)

    up8 = Conv2D(16, (2, 2), activation = 'relu', padding = 'same')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(16, (3, 3), activation = 'relu', padding = 'same')(merge8)
    conv8 = Conv2D(16, (3, 3), activation = 'relu', padding = 'same')(conv8)

    up9 = Conv2D(8, (2, 2), activation = 'relu', padding = 'same')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(8, (3, 3), activation = 'relu', padding = 'same')(merge9)
    conv9 = Conv2D(8, (3, 3), activation = 'relu', padding = 'same')(conv9)
    
    outputs = Conv2D(1, (1,1), activation = 'sigmoid')(conv9)
    
    model = Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer=Adam(lr = 1e-4), loss=bce_dice_loss, metrics=[dice_coef])
    
    return model

model_checkpoint = ModelCheckpoint("model4.hdf5", save_best_only=True, verbose=1, monitor='dice_coef', mode='max')

In [ ]:
def load_image(img_path):
      
    with rasterio.open(img_path, "r", driver='JP2OpenJPEG') as src:
        img = src.read()
        img = reshape_as_image(img)

    return img

def load_mask(img_path):
      
    with rasterio.open(img_path, "r", driver='JP2OpenJPEG') as src:
        img = src.read()
        img = reshape_as_image(img)
        img[img>1]=1
        
    return img

In [ ]:
#generate X

In [ ]:
#generate Y

In [ ]:
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_loss(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = y_true_f * y_pred_f
    score = (2. * K.sum(intersection) + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return 1. - score

def bce_dice_loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)

In [ ]:
#set model

In [ ]:
#train model

In [ ]:
#load model weights

In [ ]:
#check any name from images

In [ ]:
#visualize image

In [ ]:
#visualize mask

In [ ]:
#visualize prediction